# Problema 3 - Análisis Datos Youtube - Archivo .py

1. Descargar un archivo .zip mediante código del siguiente url (https://netsg.cs.sfu.ca/youtubedata/) (recomiendo descargar el archivo 0333.zip que es menos pesado)
2. Descomprimir los datos en una carpeta que genere y leer mediante pandas alguno de los archivos en esta. (observar que no es necesario en un primer momento leer los datos con un nombre de columna especifico)

    - Los nombres de columna pueden ser puestos posteriormente
    - El separador de columna es <code>\t</code>
    - Se colocan los nombres de columnas y descripción asociada para su intermetación. Ejemplo columna1 sera VideoID ... 
    

3. Procesar los datos según: 
    - Nos quedaremos con las columnas: VideoID, edad, catgoria, views, rate.
    - Realizar un filtrado básico a los datos. Ejemplo solo seleccionar cierto grupo de categorias

4. Procesamiento en Mongo Db
    - Exportar los datos a mongo DB 
    - Crear 2 graficos con los datos 
    - Compartir link donde encontrar los datos 




| Nombre de la Columna | Descripción                                                                                                 |
|----------------------|-------------------------------------------------------------------------------------------------------------|
| `video ID`           | Una cadena de 11 dígitos, la cual es única                                                                |
| `uploader`           | Una cadena con el nombre de usuario del cargador del video                                                  |
| `age`                | Un número entero que representa los días transcurridos desde la fecha en que se subió el video hasta el 15 de febrero de 2007 (fecha de creación de YouTube) |
| `category`           | Una cadena que indica la categoría del video elegida por el cargador                                       |
| `length`             | Un número entero que representa la duración del video en minutos                                            |
| `views`              | Un número entero que representa el número de visualizaciones del video                                      |
| `rate`               | Un número flotante que indica la calificación del video                                                      |
| `ratings`            | Un número entero que representa el número de calificaciones recibidas por el video                          |
| `comments`           | Un número entero que indica el número de comentarios en el video                                            |
| `related IDs`        | Hasta 20 cadenas de texto con los IDs de videos relacionados                                                |


In [ ]:
import os
import pandas as pd
import zipfile
from pymongo import MongoClient
import matplotlib.pyplot as plt

zip_file_path = 'datos_videos.zip'  
output_folder = 'video_data'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

dataframes = []
for i in range(4):  
    file_path = os.path.join(output_folder, f"{i}.txt")
    df = pd.read_csv(file_path, sep='\t', header=None)  
    dataframes.append(df)

df_all = pd.concat(dataframes, ignore_index=True)

df_all.columns = ['VideoID', 'Uploader', 'Age', 'Category', 'Length', 'Views', 'Rate', 'Ratings', 'Comments', 'Related_IDs']

df_filtered = df_all[['VideoID', 'Age', 'Category', 'Views', 'Rate']]

df_filtered = df_filtered[df_filtered['Category'].isin(['Music', 'Entertainment'])]

client = MongoClient('mongodb://localhost:27017/')
db = client['video_db']
collection = db['videos']

data_dict = df_filtered.to_dict('records')
collection.insert_many(data_dict)

plt.figure(figsize=(10, 6))
plt.bar(df_filtered['Category'], df_filtered['Views'], color='blue')
plt.title('Número de Visualizaciones por Categoría')
plt.xlabel('Categoría')
plt.ylabel('Número de Visualizaciones')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('views_by_category.png')

plt.figure(figsize=(10, 6))
plt.scatter(df_filtered['Age'], df_filtered['Rate'], color='red')
plt.title('Calificación vs Edad del Video')
plt.xlabel('Edad (días)')
plt.ylabel('Calificación')
plt.tight_layout()
plt.savefig('rating_vs_age.png')

print("Los datos han sido exportados a MongoDB.")
print("Gráficos generados: views_by_category.png y rating_vs_age.png")
